# Importing The Dataset

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
original_data = pd.read_csv(r"C:\Users\alisa\Downloads\Prayagraj_Traffic_Data.csv")
data = original_data.copy()

In [2]:
print(data.shape)
data.head(25)

(10080, 21)


,record_id,timestamp,intersection_id,intersection_name,latitude,longitude,vehicle_count,average_speed,signal_cycle_time,green_time,yellow_time,red_time,weather_condition,day_of_week,is_holiday,event,pollution_level,car_count,bus_count,truck_count,motorcycle_count
0,9926.0,2023-01-07 21:20:00,6.0,Allahabad Junction,25.4358,81.8307,75.0,39.72,120.0,63.0,4.0,53.0,Clear,Saturday,0.0,NaN,1.21,48.0,4.0,6.0,17.0
1,8595.0,2023-01-06 23:10:00,5.0,Naini Bridge,25.4088,81.8592,139.0,30.33,120.0,51.0,4.0,65.0,Clear,Friday,0.0,NaN,2.86,89.0,13.0,17.0,20.0
2,673.0,2023-01-01 11:10:00,3.0,Chowk,25.4314,81.8437,20.0,53.28,120.0,69.0,5.0,46.0,Clear,Sunday,1.0,NaN,0.34,15.0,1.0,2.0,2.0
3,8744.0,2023-01-07 01:40:00,4.0,Tagore Town,25.4675,81.8867,21.0,36.06,120.0,59.0,4.0,57.0,Heavy Rain,Saturday,0.0,NaN,0.54,12.0,4.0,2.0,3.0
4,316.0,2023-01-01 05:10:00,6.0,Allahabad Junction,25.4358,81.8307,41.0,47.21,120.0,NaN,5.0,54.0,Clear,Sunday,1.0,NaN,0.86,26.0,6.0,2.0,7.0
5,1092.0,2023-01-01 18:10:00,2.0,Katra,25.4550,81.8463,177.0,24.71,120.0,77.0,5.0,38.0,Clear,Sunday,1.0,New Year Celebration,3.46,111.0,18.0,15.0,33.0
6,3503.0,2023-01-03 10:20:00,3.0,Chowk,25.4314,81.8437,133.0,23.87,120.0,NaN,6.0,47.0,Heavy Rain,Tuesday,0.0,NaN,2.78,77.0,16.0,12.0,28.0
7,7477.0,2023-01-06 04:30:00,7.0,Phaphamau,25.4945,81.8686,162.0,25.93,120.0,79.0,6.0,35.0,Clear,Friday,0.0,NaN,3.16,95.0,14.0,18.0,35.0
8,2955.0,2023-01-03 01:10:00,5.0,Naini Bridge,25.4088,81.8592,32.0,30.79,120.0,60.0,5.0,55.0,Heavy Rain,Tuesday,0.0,NaN,0.77,19.0,4.0,5.0,4.0
9,6620.0,2023-01-05 14:10:00,10.0,Minto Park,25.4352,81.8820,75.0,39.93,120.0,53.0,6.0,61.0,Cold Wave,Thursday,1.0,NaN,1.86,33.0,11.0,11.0,20.0


In [3]:
data.isnull().sum()

record_id              98
timestamp              97
intersection_id        92
intersection_name     117
latitude              106
longitude             116
vehicle_count         112
average_speed          74
signal_cycle_time     108
green_time             92
yellow_time            89
red_time              102
weather_condition      78
day_of_week            94
is_holiday            104
event                9220
pollution_level       105
car_count              97
bus_count              95
truck_count           106
motorcycle_count      100
dtype: int64

# Data Cleaning

We will Perform the data cleaning process column by column so that the code doesn't get too messy

## Record ID And TimeStamp

We will assign random ID's to  recordID columns and change the timestamp column into data-Time column so that which we can use it further analysis and feature extraction Timestamp column contain some null values  but we can't do much about it so we will just drop them

In [4]:
# Record ID
def generate_unique_random_id(existing_ids, low=1, high=10000):
    while True:
        random_id = np.random.randint(low, high)
        if random_id not in existing_ids: 
            return random_id

existing_ids = set(data['record_id'].dropna()) 
data['record_id'] = data['record_id'].apply(
    lambda x: generate_unique_random_id(existing_ids) if pd.isnull(x) else x
)

# TimeStamp
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.dropna(subset = 'timestamp',inplace = True)

## Intersection ID and Intersection_name

Both of the columns contain null values so we will make a mapping between them and fill the null values according to the map
for eg if intersection_id have a null value and corrosponding intersection_name is Katra then we will fill it with 2

In [18]:
area_to_id_map = {
    'Allahabad Junction': 6, 'Naini Bridge': 5, 'Chowk': 3, 'Tagore Town': 4,
    'Katra': 2, 'Phaphamau': 7, 'Minto Park': 10, 'MNNIT Gate': 9, 
    'Civil Lines': 1, 'Jhunsi': 8
}

data['intersection_id'] = data['intersection_name'].map(area_to_id_map).fillna(data['intersection_id'])
data['intersection_name'] = data['intersection_id'].map({v: k for k, v in area_to_id_map.items()}).fillna(data['intersection_name'])

## Latitude and Longitude

We will do the same as we did above because it is also dependent on intersection_name and doesn't change

In [19]:
#  Latitude
area_to_latitude_map = {
    'Allahabad Junction': 25.4358, 'Naini Bridge': 25.4088, 'Chowk': 25.4314, 'Tagore Town': 25.4675,
    'Katra': 25.4550, 'Phaphamau': 25.4945, 'Minto Park': 25.4352, 'MNNIT Gate': 25.4920, 
    'Civil Lines': 25.4486, 'Jhunsi': 25.4326
}

data['latitude'] = data['intersection_name'].map(area_to_latitude_map).fillna(data['latitude'])

# Longitude
a_to_longitude_map = {
    'Allahabad Junction': 81.8307, 'Naini Bridge': 81.8592	, 'Chowk': 81.8437, 'Tagore Town': 81.8867,
    'Katra': 81.8463, 'Phaphamau': 81.8686, 'Minto Park': 81.882, 'MNNIT Gate': 81.8639, 
    'Civil Lines': 81.8333, 'Jhunsi': 81.9061
}

data['longitude'] = data['intersection_name'].map(city_to_longitude_map).fillna(data['longitude'])

 ## Vehicle Count , Bus Count  , Pollution Level , Car Count  , Truck Count  , Motorcycle Count

Missing values for specific columns in the data by using the mean values of these columns for each intersection
Missing values of Average will just be replaced by mean

In [7]:
# Vehicle Count
mean_values = data.groupby('intersection_name')[['vehicle_count' , 'pollution_level', 'car_count', 'bus_count', 'truck_count', 'motorcycle_count']].mean()

def fill_with_city_average(row):
    if pd.isnull(row['pollution_level']):
        row['pollution_level'] = mean_values.loc[row['intersection_name'], 'pollution_level']
    if pd.isnull(row['car_count']):
        row['car_count'] = mean_values.loc[row['intersection_name'], 'car_count']
    if pd.isnull(row['vehicle_count']):
        row['vehicle_count'] = mean_values.loc[row['intersection_name'], 'vehicle_count']        
    if pd.isnull(row['bus_count']):
        row['bus_count'] = mean_values.loc[row['intersection_name'], 'bus_count']
    if pd.isnull(row['truck_count']):
        row['truck_count'] = mean_values.loc[row['intersection_name'], 'truck_count']
    if pd.isnull(row['motorcycle_count']):
        row['motorcycle_count'] = mean_values.loc[row['intersection_name'], 'motorcycle_count']
    return row
data = data.apply(fill_with_city_average, axis=1)

# Average Speed
data['average_speed'] = data['average_speed'].fillna(data['average_speed'].mean())

## Signal_cycle_time ,green , yellow ,red time

Signal_cycle_time is constant so we will replace it with the constant value and we will treat missing values of other columns by there mean

In [8]:
data['signal_cycle_time'] = data['signal_cycle_time'].fillna(120.0)
col = [ 'green_time', 'yellow_time', 'red_time']
for c in col:
    data[c] = data[c].fillna(data[c].mean())

## Weather Condition and Event

We will replace the missing value with just unknown 

In [9]:
data['weather_condition'].fillna("unknown" , inplace = True)
data['event'].fillna("unknown" , inplace = True)

## Day of Week and isHoliday

We will extract day of the week from timestamp as it is a date - time column so we can extract from it and fill the missing values
And if it is Sunday we will assign holiday to true

In [10]:
data['day_of_week'] = data['timestamp'].dt.day_name()
data['is_holiday'] = data['day_of_week'].map({"Sunday" : 1 }).fillna(data['is_holiday'])
data['is_holiday'].fillna(0 , inplace = True)
data.dropna(inplace = True)

## Outlier Detection and Removal

In [11]:
columns_to_check = ['average_speed', 'vehicle_count']
Q1 = data[columns_to_check].quantile(0.30)
Q3 = data[columns_to_check].quantile(0.80)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = ((data[columns_to_check] < lower_bound) | (data[columns_to_check] > upper_bound))
outlier_data = data[outliers.any(axis=1)]
outlier_data.shape

(352, 21)

In [12]:
data = data[~((data[columns_to_check] < lower_bound) | (data[columns_to_check] > upper_bound)).any(axis=1)]

## Changing categorical columns into numeric columns

In [13]:
data_without_categorical_encoding = data.copy()

In [14]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
cols = ['weather_condition','event','day_of_week']
for col in cols:
    data[col] = encode.fit_transform(data[col])

data['is_holiday'] = pd.to_numeric(data['is_holiday'], errors='coerce')
data_without_categorical_encoding['is_holiday'] = pd.to_numeric(data_without_categorical_encoding['is_holiday'], errors='coerce')

# Data Transformation and Feature Engineering

Adding two new feature which will tell us that if it is a weekend or not and what hour of day is it

In [15]:
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x in [6, 7] else 0)

data['hour_of_day'] = pd.to_datetime(data['timestamp']).dt.hour
data_without_categorical_encoding['hour_of_day'] = pd.to_datetime(data_without_categorical_encoding['timestamp']).dt.hour

## Saving the Cleaned and Preprocessed Data

In [16]:
data_without_categorical_encoding.to_csv(r"C:\Users\alisa\OneDrive\Desktop\DataVision\Cleaned_Data\data_without_categorical_encoding.csv", index=False)

In [17]:
data.to_csv(r"C:\Users\alisa\OneDrive\Desktop\DataVision\Data_with_categorical_encoding_no.csv", index=False)